Elizabeth Avery Frankenberg; 
Machine Learning; 
Homework 1: NB Classification; 
January 15 2019

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("hw1_trainingset.csv")

data_labels = data['Label']
data_features = data[['Feature_1', 'Feature_2', 'Feature_3', 'Feature_4', 'Feature_5', 'Feature_6']]
data_subset = data[['Feature_1', 'Feature_2', 'Feature_4', 'Feature_5', 'Feature_6']]
data_subset1 = data[['Feature_2', 'Feature_3', 'Feature_4', 'Feature_6']]
#data_subset.head()

#data_labels.head()

After loading in the data I tried to determine which features were more important to include in the model training and fitting. First I used the VarianceThreshold feature selection method. From this measure we find no features changed and therefore were unable to eliminate the inclusion of any features.

In [205]:
from sklearn.feature_selection import VarianceThreshold
X = data_features
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(X)
#print X

array([[ 7.43014e+03,  9.52978e+03, -2.45333e+03,  1.90000e+01,
         1.23000e+02,  6.21000e+02],
       [ 1.12564e+04,  5.04551e+04, -4.22000e+03,  1.80000e+01,
         2.16000e+02,  2.67700e+03],
       [ 1.30930e+04,  5.18971e+04, -2.88000e+03,  3.00000e+01,
         2.34000e+02,  2.46400e+03],
       ...,
       [ 1.57935e+04,  2.16990e+05, -9.81900e+03,  4.40000e+01,
         3.38000e+02,  8.32300e+03],
       [ 8.78716e+03,  2.07786e+04, -3.47008e+03,  9.20000e+01,
         1.63000e+02,  1.16200e+03],
       [ 8.32240e+03,  1.22089e+04, -2.59259e+03,  8.30000e+01,
         1.28000e+02,  6.93000e+02]])

Next, I tried to use the SelectKBest feature selection algorithm. I ran this with different k values (from 1-5 because you can't include negative numbers in this test) it looks as though the features rank in this order of importance--excluding Feature 3 becuase it has negative numbers. I then used this ranking to test out different combinations of features in the training and testing.
Best to worst: 
Feature 2
Feature 6
Feature 1
Feature 5
Feature 4

In [210]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
X, y = data_subset, data_labels
X_new = SelectKBest(chi2, k=4).fit_transform(X, y)
#print X_new

Here I trained the classifier on all of the data to figure out which NB classifier (Bernoulli or Gaussian) I wanted to use and which set of features performed best. I tested these by using a method to calculate the accuracy of the predicted values. 

Here are some of the feature sets and accuracy scores that I obtained:

.594821 with all features

.59697 with features 1, 2, 4, 5, 6

.598 with features 2, 6

.599 with features 2, 3, 6

.601 with features 2, 3, 4, 5, 6

.608 with features 2, 3, 4, 6 

After doing this I trained the classifier on all of the data looking at the difference between Bernoulli and Gaussian NB. 


In [85]:
#Training the classifier on all of the data
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
model_B = BernoulliNB()
model_G = GaussianNB()
#training, fitting and learning
model_B.fit(data_subset, data_labels)
model_G.fit(data_subset1, data_labels)

GaussianNB(priors=None)

In [86]:
#1 the predicted model
predicted = model_G.predict(data_subset1)
#the length of the target array
lenTarArr = len(data_labels)
correct = 0
#for each element 
for i in range(lenTarArr):
    #if the binary number at position i is equal to the item in that spot in the target array, then increment correct
    if data_labels[i] == predicted[i]:
        correct += 1
#taking the number of correct values out of the total number
print('accuracy', float(correct)/lenTarArr)

('accuracy', 0.6080357142857142)


I found that Gaussian performed better than Bernoulli but I wanted to check and see what Complement and Multinomial would do as well. Therefore, I made this function to calculate the accuracy with different features with different methods. I actually found that after doing the absolute value of the data set, so that there were no negative numbers, I was able to get a better accuracy with Multinomial NB. 

In [32]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB, ComplementNB, MultinomialNB, BernoulliNB 
def try_all_features():
    data = pd.read_csv("hw1_trainingset.csv")
    data_labels = data['Label']
    data_features = data[['Feature_1', 'Feature_2', 'Feature_3', 'Feature_4', 'Feature_5', 'Feature_6']]
    Y = np.array(data_labels)
    best_score= 0
    features = None
    
    #this transforms the data so there are negatives
#     data_features = data_features.apply(lambda x:x+10000000)
    data_features = data_features.abs()
    
    for i in range(0, 6):
        for j in range(0, 6):
            
            if (j < i):
                data_new = np.vstack(np.array(data_features.iloc[:,j:i]))
        
            elif (i == j):
                data_new = np.vstack(np.array(data_features.iloc[:,i]))
        
            else:
                data_new = np.vstack(np.array(data_features.iloc[:,i:j]))
                
        modelKFolds = MultinomialNB()

        train_scores = []
        test_scores = []
        score = 0
        kf = KFold(n_splits=10)
        for train, test in kf.split(data_new):
            x_train, y_train = data_new[train], Y[train]
            x_test, y_test = data_new[test], Y[test]
        
            modelKFolds.fit(x_train, y_train)
            y_pred = modelKFolds.predict(x_test)
            score += metrics.accuracy_score(y_test, y_pred) /10
            
        if (score > best_score):
            best_score = score
            features = data_new
            print "highest accuracy %.4f" %best_score 
            print features           

In [33]:
try_all_features()

highest accuracy 0.5723
[[7.43014e+03 9.52978e+03 2.45333e+03 1.90000e+01 1.23000e+02]
 [1.12564e+04 5.04551e+04 4.22000e+03 1.80000e+01 2.16000e+02]
 [1.30930e+04 5.18971e+04 2.88000e+03 3.00000e+01 2.34000e+02]
 ...
 [1.57935e+04 2.16990e+05 9.81900e+03 4.40000e+01 3.38000e+02]
 [8.78716e+03 2.07786e+04 3.47008e+03 9.20000e+01 1.63000e+02]
 [8.32240e+03 1.22089e+04 2.59259e+03 8.30000e+01 1.28000e+02]]
highest accuracy 0.5786
[[9.52978e+03 2.45333e+03 1.90000e+01 1.23000e+02]
 [5.04551e+04 4.22000e+03 1.80000e+01 2.16000e+02]
 [5.18971e+04 2.88000e+03 3.00000e+01 2.34000e+02]
 ...
 [2.16990e+05 9.81900e+03 4.40000e+01 3.38000e+02]
 [2.07786e+04 3.47008e+03 9.20000e+01 1.63000e+02]
 [1.22089e+04 2.59259e+03 8.30000e+01 1.28000e+02]]
highest accuracy 0.6582
[[2453.33   19.    123.  ]
 [4220.     18.    216.  ]
 [2880.     30.    234.  ]
 ...
 [9819.     44.    338.  ]
 [3470.08   92.    163.  ]
 [2592.59   83.    128.  ]]


I got the best accuracy score with Multinomial NB of 0.6582 for an individual KFold and included Feature 3, 4, and 5.  

After choosing the features I wanted to include, I then wrote the train and predict functions that implemented KFolds cross validation using MultinomialNB after transforming the values to be the absolute value. I decided to test out a few more variations when I actually ran this train method and was able to get a higher confidence score using Features 1, 3, 4 and 5. Here are these methods:

In [20]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB

def train(filename):
    
    data = pd.read_csv(filename)
    data_labels = data['Label']
    data_features = data[['Feature_1', 'Feature_3', 'Feature_4', 'Feature_5']]
    #this transforms the data so there are negatives
#     data_features = data_features.apply(lambda x:x+10000000)
    data_features = data_features.abs()
    
    accuracy = 0
    modelKFolds = MultinomialNB()

    train_scores = []
    test_scores = []

    kf = KFold(n_splits=10)
    X = np.array(data_features)
    print X
    Y = np.array(data_labels)
    print Y
    for train, test in kf.split(X):
        x_train, y_train = X[train], Y[train]
        x_test, y_test = X[test], Y[test]
        
        modelKFolds.fit(x_train, y_train)
        y_pred = modelKFolds.predict(x_test)
        score = metrics.accuracy_score(y_test, y_pred)
        accuracy += score
    
    
    print score
    return modelKFolds   

In [24]:
def predict(C, row):
    row = row.reshape(1, -1)
    prediction = C.predict(row)
    return prediction

In [27]:
import csv
import numpy as np

def predict_file(C, feature_test_filename, predict_test_filename):
    test_data = pd.read_csv(feature_test_filename)
    with open(predict_test_filename, 'w') as writeFile:
        writer = csv.writer(writeFile) 
        data_features = test_data[['Feature_1', 'Feature_3', 'Feature_4', 'Feature_5']]
        writer.writerow(['Feature_1', 'Feature_3', 'Feature_4', 'Feature_5', 'Label'])
        X = np.array(data_features)
        line = 0
        while (line < 2400):
            row = X[line]
            prediction = predict(C, row)
            row = np.append(row, prediction)
            writer.writerow(row)
            line += 1
        

After writing these methods, I then called train on the training set and predict on the test set with the new predictions included. The training set got an accuracy of 0.69 which is far improved from the initial 0.608 I was getting at the start. The predict_file method also generated a new file with the labels in it which I will be turning in!

In [30]:
C = train("hw1_trainingset.csv")

[[ 7430.14  2453.33    19.     123.  ]
 [11256.4   4220.      18.     216.  ]
 [13093.    2880.      30.     234.  ]
 ...
 [15793.5   9819.      44.     338.  ]
 [ 8787.16  3470.08    92.     163.  ]
 [ 8322.4   2592.59    83.     128.  ]]
[0 0 0 ... 1 0 0]
0.6928571428571428


In [31]:
predict_file(C, "hw1_testset.csv", "hw1_testset_pred.csv")